In [1]:
!pip install beautifulsoup4
!pip install requests

from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml.html as lh

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [3]:
#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ("%d: %s" %(i,name))
    col.append((name,[]))
    

1: Postcode
2: Borough
3: Neighbourhood



In [4]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
        
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
       
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        

In [5]:
[len(C) for (title,C) in col]

[287, 287, 287]

In [6]:
Dict={title:column for (title,column) in col}
df = pd.DataFrame(Dict)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [7]:
df = df[df.Borough!='Not assigned']
df = df.replace('\n','',regex=True)
df.rename(columns={"Neighbourhood\n": "Neighbourhood"}, inplace=True)
df.reset_index(drop=True, inplace=True)

In [8]:
df = df.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(x))

In [10]:
df.head()

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Downtown Toronto,Queen's Park


In [ ]:
import pandas as pd

df_geo = pd.read_csv("https://cocl.us/Geospatial_data")

In [ ]:
df = pd.merge(df, df_geo, left_on='Postcode', right_on='Postal Code').drop(['Postal Code'], axis=1)